# Looking at YouTube view count growth for top MVs 

## Strategy: 

1. We are going to write two scripts. One is going to get the url and title of the MV. Anotehr is going to write a script to collect the views of the music video from a specific time period. This should also contain the url. 
2. We are going to look at the top 5 trending on Youtube. 

In [1]:
%load_ext autoreload
%autoreload 2

## Importing our libraries 

In [6]:
import requests
import random
import time
from selenium.webdriver import Chrome
# from scrape_functions import (
#     get_comic_title,
#     find_image,
#     scrape_image,
#     get_prev)

In [19]:
browser = Chrome()
url = "https://www.youtube.com/results?search_query=%23music+video"
browser.get(url)

In [4]:
# title = browser.find_element_by_id('video-title')

In [5]:
import requests

def get_mv_title(browser):
    """Return the title text"""
    title = browser.find_element_by_id('video-title')
    return title

In [6]:
def get_mv_url(title):
    """Return the url text"""
    return title.get_attribute('href')

In [21]:
from datetime import datetime

def get_view_count(browser, url):
    browser.get(url)
    time.sleep(3)
    """Return the view count"""
    today = datetime.now()
    sel = 'span.view-count'
    view_count = browser.find_element_by_css_selector(sel).text
    return int(view_count.split()[0].replace(",","")), today

In [14]:
count, today = get_view_count(browser, 'https://www.youtube.com/watch?v=BQ0mxQXmLsk')

In [15]:
count

763375054

In [24]:
url = "https://www.youtube.com/results?search_query=%23music+video"
browser.get(url)
time.sleep(5)
videolinks = browser.find_elements_by_id('video-title')
videos = [(link.text, link.get_attribute('href')) for link in videolinks]

info = []
for title, url in videos:
#     title = get_mv_title(video)
    view_count, today = get_view_count(browser, url)
    info.append({'title': title,
                 'url': url,
                 'view_count': view_count,
                 'today': today})

KeyboardInterrupt: 

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(info)

In [ ]:
df.head()

## Saving the file 

In [121]:
from pymongo import MongoClient 
from odo import odo

# open connection
connection = MongoClient()

# get db
db = connection.music_scraping.


# insert df into <table> of <dataframe>
odo(DF, db.<table>)

SyntaxError: invalid syntax (<ipython-input-121-cf31e6748725>, line 8)